<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"></li></ul></li><li><span><a href="#Проект-для-«Викишоп»" data-toc-modified-id="Проект-для-«Викишоп»-1">Проект для «Викишоп»</a></span><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1.1">Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1.1">Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-1.2">Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-1.2.0.1">Логистическая регрессия</a></span></li><li><span><a href="#Модель-решающего-дерева" data-toc-modified-id="Модель-решающего-дерева-1.2.0.2">Модель решающего дерева</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-1.2.0.3">Модель случайного леса</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-1.2.0.4">LinearSVC</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2.1">Вывод</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-1.3">Тестирование</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.3.1">Вывод</a></span></li></ul></li><li><span><a href="#Основные-результаты" data-toc-modified-id="Основные-результаты-1.4">Основные результаты</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-1.5">Чек-лист проверки</a></span></li></ul></li></ul></div>

# Проект для интернет-магазина

Интернет-магазин запускает новый сервис: пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. Имеется набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

Импорт библиотек, которые потребуются для выполнения исследования:

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import make_scorer, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords 
import nltk
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings('ignore')

Загрузка датасета:

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')

Вывод первых строк датасета:

In [3]:
pd.set_option('display.max_colwidth', 100)
toxic_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


Общая информация о данных таблицы **toxic_comments**:

In [4]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Датасет состоит из двух столбцов. В столбце **text** содержится текст комментария, а в столбце **toxic** — целевой признак (положительно или отрицательно окрашен комментарий).

Общее количество объектов - 159 571, пропусков нет, типы данных корректны.

---

Подготовим исходные данные для дальнейшего исседования и построения прогноза.

Убедимся, что в столбце целевых признаков данные принимают только два значения:

In [5]:
display(toxic_comments['toxic'].value_counts())
display(toxic_comments['toxic'].value_counts(normalize = True))

0    143346
1     16225
Name: toxic, dtype: int64

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Признак разбивает датасет на группы в пропорции примерно 90:10. Вообще говоря, по условию не сказано, в какой группе содержатся негативные комментарии, однако если взглянуть на несколько строчек датасета, это становится очевидно:

In [6]:
toxic_comments.query('toxic == 1').head()

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are ...,1
16,"Bye! \n\nDon't look, come or think of comming back! Tosser.",1
42,"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are ...",1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1


Чтобы комментарии можно было далее анализировать, очистим их от посторонних символов, оставив только латиницу, и приведем все слова к нижнему регистру:

In [7]:
#заменяет все символы, кроме латинских букв, на пробелы, а затем объединяет в строку через пробел, удаляя лишние
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    return " ".join(text.split())

In [8]:
toxic_comments['only_letters'] = toxic_comments['text'].apply(lambda x: clear_text(x.lower()))
toxic_comments.head()

,text,toxic,only_letters
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,explanation why the edits made under my username hardcore metallica fan were reverted they weren...
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,d aww he matches this background colour i m seemingly stuck with thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,hey man i m really not trying to edit war it s just that this guy is constantly removing relevan...
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,more i can t make any real suggestions on improvement i wondered if the section statistics shoul...
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that s on


Разделим комментарии на токены по пробелу:

In [9]:
toxic_comments['tokenized'] = toxic_comments['only_letters'].apply(lambda x: re.split(' ', x))
toxic_comments.head()

,text,toxic,only_letters,tokenized
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,explanation why the edits made under my username hardcore metallica fan were reverted they weren...,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, revert..."
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,d aww he matches this background colour i m seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, m, seemingly, stuck, with, thanks, talk, janu..."
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,hey man i m really not trying to edit war it s just that this guy is constantly removing relevan...,"[hey, man, i, m, really, not, trying, to, edit, war, it, s, just, that, this, guy, is, constantl..."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,more i can t make any real suggestions on improvement i wondered if the section statistics shoul...,"[more, i, can, t, make, any, real, suggestions, on, improvement, i, wondered, if, the, section, ..."
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that s on,"[you, sir, are, my, hero, any, chance, you, remember, what, page, that, s, on]"


Уберем стоп-слова:

In [10]:
stop_words = nltk.corpus.stopwords.words('english')

In [11]:
def delete_stopwords(text):
    text = [word for word in text if word not in stop_words]
    return text

In [12]:
toxic_comments['stopwords_free'] = toxic_comments['tokenized'].apply(lambda x: delete_stopwords(x))
toxic_comments.head()

,text,toxic,only_letters,tokenized,stopwords_free
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,explanation why the edits made under my username hardcore metallica fan were reverted they weren...,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, revert...","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalisms, closure, ga..."
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,d aww he matches this background colour i m seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, m, seemingly, stuck, with, thanks, talk, janu...","[aww, matches, background, colour, seemingly, stuck, thanks, talk, january, utc]"
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,hey man i m really not trying to edit war it s just that this guy is constantly removing relevan...,"[hey, man, i, m, really, not, trying, to, edit, war, it, s, just, that, this, guy, is, constantl...","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking,..."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,more i can t make any real suggestions on improvement i wondered if the section statistics shoul...,"[more, i, can, t, make, any, real, suggestions, on, improvement, i, wondered, if, the, section, ...","[make, real, suggestions, improvement, wondered, section, statistics, later, subsection, types, ..."
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that s on,"[you, sir, are, my, hero, any, chance, you, remember, what, page, that, s, on]","[sir, hero, chance, remember, page]"


Лемматизируем оставшиеся слова:

In [13]:
lemmatizer = WordNetLemmatizer()

In [14]:
toxic_comments['lemmatized'] = \
            toxic_comments['stopwords_free'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x]))
toxic_comments.head()

,text,toxic,only_letters,tokenized,stopwords_free,lemmatized
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,explanation why the edits made under my username hardcore metallica fan were reverted they weren...,"[explanation, why, the, edits, made, under, my, username, hardcore, metallica, fan, were, revert...","[explanation, edits, made, username, hardcore, metallica, fan, reverted, vandalisms, closure, ga...",explanation edits made username hardcore metallica fan reverted vandalism closure gas voted new ...
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,d aww he matches this background colour i m seemingly stuck with thanks talk january utc,"[d, aww, he, matches, this, background, colour, i, m, seemingly, stuck, with, thanks, talk, janu...","[aww, matches, background, colour, seemingly, stuck, thanks, talk, january, utc]",aww match background colour seemingly stuck thanks talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,hey man i m really not trying to edit war it s just that this guy is constantly removing relevan...,"[hey, man, i, m, really, not, trying, to, edit, war, it, s, just, that, this, guy, is, constantl...","[hey, man, really, trying, edit, war, guy, constantly, removing, relevant, information, talking,...",hey man really trying edit war guy constantly removing relevant information talking edits instea...
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,more i can t make any real suggestions on improvement i wondered if the section statistics shoul...,"[more, i, can, t, make, any, real, suggestions, on, improvement, i, wondered, if, the, section, ...","[make, real, suggestions, improvement, wondered, section, statistics, later, subsection, types, ...",make real suggestion improvement wondered section statistic later subsection type accident think...
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir are my hero any chance you remember what page that s on,"[you, sir, are, my, hero, any, chance, you, remember, what, page, that, s, on]","[sir, hero, chance, remember, page]",sir hero chance remember page


### Вывод

Исходный датасет состоит из двух столбцов: **text** с текстом комментариеви и **toxic** — положительной или отрицательной окраской комментария.

Токсичных комментариев примерно 10% от общего количества.

Комментарии очищены от всех символов (за исключением латинских букв), стоп-слов, а затем лемматизированы.

## Обучение

Разделим данные на тестовую, валидационную и обучающую выборки в соотношении 70:15:15.

In [15]:
features = toxic_comments['lemmatized']
target = toxic_comments['toxic']

In [16]:
features_train, features_valid, target_train, target_valid = \
                train_test_split(features, target, test_size = 0.3, random_state = 42, stratify = target)
features_valid, features_test, target_valid, target_test = \
                train_test_split(features_valid, target_valid, test_size = 0.5, random_state = 42, stratify = target_valid)

print('Размер обучающей выборки:', features_train.shape, target_train.shape)
print('Размер валидационной выборки:', features_valid.shape, target_valid.shape)
print('Размер тестовой выборки:', features_test.shape, target_test.shape)

Размер обучающей выборки: (111699,) (111699,)
Размер валидационной выборки: (23936,) (23936,)
Размер тестовой выборки: (23936,) (23936,)


Вычислим **TF-IDF**:

In [17]:
count_tf_idf = TfidfVectorizer()
features_train_tf_idf = count_tf_idf.fit_transform(features_train) 
features_valid_tf_idf = count_tf_idf.transform(features_valid) 

Функция для поиска оптимального набора гиперпараметров моделей по метрике F1:

In [18]:
def grid_search(model, params, features, target):
    
    f1 = make_scorer(f1_score, greater_is_better = True)
    g_search = GridSearchCV(model, params, scoring = f1, n_jobs = -1)
    g_search.fit(features, target)
    
    print('Лучший набор гиперпараметров:', g_search.best_params_)
    print('Качество предсказания:', round(g_search.best_score_, 5)) 
    print('Лучшая модель:', g_search.best_estimator_)
    return g_search.best_estimator_

Функция для оценки качества результатов работы модели:

In [19]:
# обучает модель на тренировочной выборке и оценивает качество метрикой F1 на тестовой, результаты записывает в таблицу.

def model_result(model, f_train, f_test, t_train, t_test, df_result):

    model.fit(f_train, t_train)
    predicted = model.predict(f_test)

    f1 = f1_score(t_test, predicted).round(5)
    print('Качество предсказания на тестовой выборке:', f1)
    
    df_result.append({'model': model.__class__.__name__, 'F1': f1})

Обучим несколько разных моделей и оценим их качество на тестовой выборке. Записывать результаты будем в отдельную таблицу.

In [20]:
result = []

#### Логистическая регрессия

В качестве гиперпараметра выберем значение обратной регуляризации:

In [21]:
lr_model = LogisticRegression(random_state = 42, solver = 'liblinear', class_weight = 'balanced')
lr_params = {'C' : range(1, 10, 1)}

In [22]:
%%time
lr_best = grid_search(lr_model, lr_params, features_train_tf_idf, target_train)

Лучший набор гиперпараметров: {'C': 7}
Качество предсказания: 0.76843
Лучшая модель: LogisticRegression(C=7, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)
CPU times: user 4min 41s, sys: 4min 1s, total: 8min 42s
Wall time: 8min 43s


In [23]:
model_result(lr_best, features_train_tf_idf, features_valid_tf_idf, target_train, target_valid, result)

Качество предсказания на тестовой выборке: 0.76552


In [24]:
pd.DataFrame(result)

,model,F1
0,LogisticRegression,0.76552


#### Модель решающего дерева

В качестве гиперпараметра возьмем глубину дерева:

In [25]:
dt_model = DecisionTreeClassifier(random_state = 42, class_weight = 'balanced')
dt_params = {'max_depth' : range(4, 10, 1)}

In [26]:
dt_best = grid_search(dt_model, dt_params, features_train_tf_idf, target_train)

Лучший набор гиперпараметров: {'max_depth': 8}
Качество предсказания: 0.56761
Лучшая модель: DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=8,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')


In [27]:
model_result(dt_best, features_train_tf_idf, features_valid_tf_idf, target_train, target_valid, result)

Качество предсказания на тестовой выборке: 0.549


In [28]:
pd.DataFrame(result)

,model,F1
0,LogisticRegression,0.76552
1,DecisionTreeClassifier,0.54900


Как видно, ни на тестовой, ни на валидационной выборке модель не справилась с предсказанием с нужной точностью.

#### Модель случайного леса

В качестве гиперпараметров будем использовать глубину дерева, а также число деревьев:

In [29]:
rf_model = RandomForestClassifier(random_state = 42, class_weight = 'balanced')
rf_params = {'max_depth' : range(1, 5, 1), 'n_estimators' : range(1, 21, 5)}

In [30]:
%%time
rf_best = grid_search(rf_model, rf_params, features_train_tf_idf, target_train)

Лучший набор гиперпараметров: {'max_depth': 3, 'n_estimators': 16}
Качество предсказания: 0.32813
Лучшая модель: RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=16, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)
CPU times: user 33.3 s, sys: 0 ns, total: 33.3 s
Wall time: 33.6 s


In [31]:
model_result(rf_best, features_train_tf_idf, features_valid_tf_idf, target_train, target_valid, result)

Качество предсказания на тестовой выборке: 0.23593


In [32]:
pd.DataFrame(result)

,model,F1
0,LogisticRegression,0.76552
1,DecisionTreeClassifier,0.54900
2,RandomForestClassifier,0.23593


Качество работы модели случайного леса даже на оптимальных параметрах значительно хуже, чем у логистической регрессии и даже дерева решений.

#### LinearSVC

Попробуем линейную модель с опорными векторами. В качестве гиперпараметра возьмем значение коэффициента регуляризации:

In [33]:
lsvc_model = LinearSVC(random_state = 42, class_weight = 'balanced', max_iter = 10000)
lsvc_params = {'C' : range(1, 5, 1)}

In [34]:
%%time
lsvc_best = grid_search(lsvc_model, lsvc_params, features_train_tf_idf, target_train)

Лучший набор гиперпараметров: {'C': 1}
Качество предсказания: 0.76132
Лучшая модель: LinearSVC(C=1, class_weight='balanced', dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)
CPU times: user 39.9 s, sys: 0 ns, total: 39.9 s
Wall time: 40.6 s


In [35]:
model_result(lsvc_best, features_train_tf_idf, features_valid_tf_idf, target_train, target_valid, result)

Качество предсказания на тестовой выборке: 0.7603


In [36]:
pd.DataFrame(result)

,model,F1
0,LogisticRegression,0.76552
1,DecisionTreeClassifier,0.54900
2,RandomForestClassifier,0.23593
3,LinearSVC,0.76030


### Вывод

Таким образом, на валидационной выборке две модели - логистическая регрессия и линейная модель опорных векторов - показали результат выше требуемого (f1 > 0.75). Модели решающего дерева и случайного леса с задачей не справились.

## Тестирование

Проверим работу моделей на тестовой выборке. Для обучения используем объединение тренировочной и валидационной выборок. В качестве гиперпараметров передадим те, что давали лучший результат на предыдущем шаге.

In [37]:
features_train = pd.concat([features_train, features_valid])
target_train = pd.concat([target_train, target_valid])  

Поскольку выборки объединены, нужно снова применить **TfidfVectorizer**:

In [38]:
count_tf_idf = TfidfVectorizer()
features_train_tf_idf = count_tf_idf.fit_transform(features_train) 
features_test_tf_idf = count_tf_idf.transform(features_test) 

In [39]:
result_test = []

In [40]:
for model in [lr_best, dt_best, rf_best, lsvc_best]:
    print(model.__class__.__name__)
    model_result(model, features_train_tf_idf, features_test_tf_idf, target_train, target_test, result_test)
    print('')

LogisticRegression
Качество предсказания на тестовой выборке: 0.76905

DecisionTreeClassifier
Качество предсказания на тестовой выборке: 0.54833

RandomForestClassifier
Качество предсказания на тестовой выборке: 0.23681

LinearSVC
Качество предсказания на тестовой выборке: 0.7624



Получили итоговую таблицу метрик на тестовой выборке:

In [41]:
pd.DataFrame(result_test).sort_values(by = 'F1')

,model,F1
2,RandomForestClassifier,0.23681
1,DecisionTreeClassifier,0.54833
3,LinearSVC,0.76240
0,LogisticRegression,0.76905


### Вывод

Результаты работы моделей на тестовой выборке не изменились: лучшее значение F1 оказалось у логистической регрессии (0,769). Также условию удовлетворяет линейная модель опорных векторов со значением метрики 0,762. Решающее дерево и случайный лес сработали неудачно, несмотря на подбор гиперпараметров.

## Основные результаты

В работе решалась задача классификации интернет-комментариев на позитивные и негативные.

В ходе предварительного анализа данных: 
- выявлен дисбаланс классов (доля токсичных комментариев в датасете составляет около 10%);
- комментарии очищены от всех символов, за исключением латинских букв;
- удалены стоп-слова;
- проведена лемматизация слов, входящих в комментарий.

Для решения задачи прогнозирования обучены и протестированы следующие модели: 
- логистической регрессии;
- дерева решений;
- случайного леса;
- LinearSVC.

Датасет разделен на тренировочную, валидационную и тестовую выборки в пропорции 70:15:15. К столбцу признаков применена векторизация с помощью *TfidfVectorizer*. В качестве метрики использовалась F1. При построении моделей учтен дисбаланс классов.

Для каждой модели подбиралиись оптимальные гиперпараметры: для логистической регрессии - коэффициент обратной регуляризации, для решающего дерева - глубина, для случайного леса - глубина и число деревьев. Для модели LinearSVC - значение коэффициента регуляризации.

Результаты работы моделей на тестовой выборке практически не изменились по сравнению с валидационной: лучшее значение F1 оказалось у логистической регрессии (0,769). Она и линейная модель опорных векторов со значением метрики 0,762 удовлетворяют условию задачи достичь значения F1 не менее 0,75. Решающее дерево и случайный лес сработали неудачно, несмотря на подбор гиперпараметров.